<a href="https://colab.research.google.com/github/harsh-agar/E-Net/blob/master/MFPT_cnn_sv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [75]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalMaxPooling2D, ConvLSTM2D, Reshape, TimeDistributed, Activation, LSTM
from keras.optimizers import RMSprop
from sklearn.preprocessing import LabelBinarizer
from scipy.io import loadmat
from keras.callbacks import ReduceLROnPlateau

from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt

In [76]:
def plotsave(history):
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validaton'], loc='upper left')
    plt.savefig('histryofloss.pdf', bbox_inches='tight')
    pass

In [77]:
dir_ds_imf1 = "/content/drive/MyDrive/final_signal.mat"

In [78]:
data = loadmat(dir_ds_imf1) # loading the data into dictionary format
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'b_type', 'c_type', 'd_type', 'final_label', 'final_signal', 'i', 'n', 'o'])


In [79]:
signalData = data['final_signal']
U_L = data['final_label']
T_L = data['final_label']

# Normalize the data
signalData = signalData / 255.0

signalData = np.reshape(signalData,(len(signalData),32,len(np.transpose(signalData)),1))

label_encoder  = LabelBinarizer()
U_L  = label_encoder.fit_transform(U_L)
T_L  = label_encoder.fit_transform(T_L)

signalData.shape

(4290, 32, 32, 1)

In [80]:
random_seed = 2

'DATA SPLITING'
X_train, X_val, Y_train, Y_val = train_test_split(signalData, T_L, test_size = 0.3, random_state=random_seed)

X_train.shape

(3003, 32, 32, 1)

In [54]:
# model = Sequential()
# #https://machinelearningmastery.com/cnn-long-short-term-memory-networks/?fbclid=IwAR1EuGy6pC-rFAt7LVPeFKZNe4oRgPftlH-V6SMl248TLlCu6KAhRKIR70s
# # model = Sequential()
# # # define CNN model
# # model.add((Conv2D(...))
# # model.add((MaxPooling2D(...)))
# # model.add((Flatten()))
# # # define LSTM model
# # model.add(LSTM(...))
# # model.add(Dense(...))

# # model = Sequential()
# # # define CNN model
# # model.add((Conv2D(...))
# # model.add((MaxPooling2D(...)))
# # model.add((Flatten()))
# # # define LSTM model
# # model.add(LSTM(...))
# # model.add(Dense(...))


# model.add((Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape = (32,32,1))))

# #tanh, elu
# model.add((Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu')))

# model.add((MaxPool2D(pool_size=(2,2))))
# model.add((Dropout(0.25)))


# model.add((Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu')))
# model.add((MaxPool2D(pool_size=(2,2), strides=(2,2))))
# model.add((Dropout(0.25)))

# # https://medium.com/@iamvarman/how-to-calculate-the-number-of-parameters-in-the-cnn-5bd55364d7ca


# model.add((Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu')))
# #model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add((Dropout(0.25)))


# model.add((Flatten()))
# model.add((Dense(128, activation = "relu")))
# model.add((Dropout(0.5)))

# # output layer
# model.add(Dense(3, activation = "softmax")) 
# # '''
# # LSTM Layers
# # '''
# # model.add(Reshape((X_train[0],32*32)))
# # model.add(keras.layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
# # model.add(keras.layers.LSTM(64))

# # # dense layer
# # model.add(keras.layers.Dense(64, activation='relu'))
# # model.add(keras.layers.Dropout(0.3))

# # # output layer
# # model.add(keras.layers.Dense(3, activation='softmax'))

#  # output softmax
# # Define the optimizer
# # https://keras.io/ko/optimizers/
# optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # <<
# # Compile the model


In [81]:
model = Sequential()

model.add(Reshape((1,32,32,1), input_shape = (32,32,1)))
# model.add(Reshape((3, 4), input_shape=(12,)))

model.add(TimeDistributed(Conv2D(filters = 32, kernel_size = (5,5), input_shape = (1,32,32,1), padding = 'Same')))
model.add(Activation('relu'))

#tanh, elu
model.add(TimeDistributed(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same')))
model.add(Activation('relu'))

model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same')))
model.add(Activation('relu'))

model.add(TimeDistributed(MaxPool2D(pool_size=(2,2), strides=(2,2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same')))
model.add(Activation('relu'))
#model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add((Dropout(0.25)))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))

# dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(3, activation='softmax'))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # <<
# Compile the model

In [73]:
model.build((32, 32, 32, 1))
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 1, 32, 32, 1)      0         
_________________________________________________________________
time_distributed_181 (TimeDi (None, 1, 32, 32, 32)     832       
_________________________________________________________________
activation_56 (Activation)   (None, 1, 32, 32, 32)     0         
_________________________________________________________________
time_distributed_182 (TimeDi (None, 1, 32, 32, 32)     9248      
_________________________________________________________________
activation_57 (Activation)   (None, 1, 32, 32, 32)     0         
_________________________________________________________________
time_distributed_183 (TimeDi (None, 1, 16, 16, 32)     0         
_________________________________________________________________
time_distributed_184 (TimeDi (None, 1, 16, 16, 32)   

In [82]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [83]:
epochs = 100 #<<100
batch_size = 32 # <<
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 1, 32, 32, 1)      0         
_________________________________________________________________
time_distributed_190 (TimeDi (None, 1, 32, 32, 32)     832       
_________________________________________________________________
activation_60 (Activation)   (None, 1, 32, 32, 32)     0         
_________________________________________________________________
time_distributed_191 (TimeDi (None, 1, 32, 32, 32)     9248      
_________________________________________________________________
activation_61 (Activation)   (None, 1, 32, 32, 32)     0         
_________________________________________________________________
time_distributed_192 (TimeDi (None, 1, 16, 16, 32)     0         
_________________________________________________________________
time_distributed_193 (TimeDi (None, 1, 16, 16, 32)   

In [ ]:
history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
          validation_data = (X_val, Y_val), verbose = 1)

Epoch 1/100
94/94 [==============================] - 36s 37ms/step - loss: 1.0632 - accuracy: 0.4104 - val_loss: 1.0817 - val_accuracy: 0.6014
Epoch 2/100
94/94 [==============================] - 2s 22ms/step - loss: 0.9204 - accuracy: 0.4714 - val_loss: 1.0562 - val_accuracy: 0.4141
Epoch 3/100
94/94 [==============================] - 2s 22ms/step - loss: 0.7782 - accuracy: 0.5105 - val_loss: 0.5637 - val_accuracy: 0.6037
Epoch 4/100
94/94 [==============================] - 2s 22ms/step - loss: 0.8042 - accuracy: 0.5796 - val_loss: 0.5830 - val_accuracy: 0.5859
Epoch 5/100
94/94 [==============================] - 2s 21ms/step - loss: 0.7053 - accuracy: 0.5772 - val_loss: 1.2407 - val_accuracy: 0.4468
Epoch 6/100
94/94 [==============================] - 2s 21ms/step - loss: 0.6973 - accuracy: 0.5836 - val_loss: 0.5685 - val_accuracy: 0.5859
Epoch 7/100
94/94 [==============================] - 2s 21ms/step - loss: 0.8365 - accuracy: 0.5863 - val_loss: 1.0243 - val_accuracy: 0.4141
Epoch

In [ ]:
#dir_saveTo = "/content/gdrive/MyDrive/BRAC/Thesis/Thesis(BRACU)/Pre-Thesis 2/Google Colab/t_05-CNN_RNN/CNN/SaveIMF1"
#model.save(dir_saveTo)

In [ ]:
plotsave(history)

In [ ]:
##################################### TEST 1  
# Predict the values from the  dataset
Y_pred = model.predict(signalData)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(U_L,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
print(confusion_mtx)
target_names = ['normal', 'ball', 'inner']
# Argmax function is deconverting from hot encoding to simple one
print (classification_report(Y_pred_classes,Y_true, target_names = target_names))

[[1716    0    0]
 [1716    0    0]
 [   0    0  858]]
              precision    recall  f1-score   support

      normal       1.00      0.50      0.67      3432
        ball       0.00      0.00      0.00         0
       inner       1.00      1.00      1.00       858

    accuracy                           0.60      4290
   macro avg       0.67      0.50      0.56      4290
weighted avg       1.00      0.60      0.73      4290



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
